## Imports

In [ ]:
# basic
import numpy as np
import random
import time
import os
import copy
import itertools

# pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data.dataset import random_split
import torchvision
from torchvision import datasets, models, transforms

# plot
import pylab
import matplotlib.pyplot as plt

# machine learning
from sklearn.metrics import confusion_matrix

# custom
import video_loader
import custom_models as cm

In [ ]:
# manual setting of clusters, as an alternative to graph inference
manual_tree = np.zeros((98,5), dtype=int)
list_ = [17, 13, 15, 16, 11, 12, 10, 14,
          3, 4, 9, 8, 6, 5, 7, 0, 1, 2,
          35, 36, 37, 38, 31, 32, 33, 34,
          30, 28, 29, 19, 20, 21, 25, 26, 27, 24, 22, 23,
          48, 46, 47, 43, 44, 45, 39, 40, 41, 18, 42]

second_list = []

for i in list_:
    second_list.append(i*2)
    second_list.append(i*2 + 1)
    
manual_tree[:,0] = second_list

manual_tree[:20,1] = 0
manual_tree[20:38,1] = 1
manual_tree[38:62,1] = 2
manual_tree[62:78,1] = 3
manual_tree[78:,1] = 4

manual_tree[:38,2] = 0
manual_tree[38:62,2] = 1
manual_tree[62:,2] = 2

manual_tree[:38,3] = 0
manual_tree[38:,3] = 1

In [ ]:
manual_tree

## Create dataloaders

See finetung.ipynb for remarks.

### CK+

In [ ]:
use_gpu = torch.cuda.is_available()

classes =  {0:'anger', 1:'contempt', 2:'disgust', 3:'fear', 4:'happy', 5:'sadness', 6:'surprise'}
N_frames = 3
N_classes = len(classes)
N_landmarks = 68

# preprocessing
data_dir = os.path.join('/home','nii','Documents','CK+')

data_transforms = transforms.Compose(
    [transforms.Resize((64,64))])

K = 10
k_folders = ['set_' + str(idx) for idx in range(K)]

    
training_datasets = {x: video_loader.VideoFolder(root=data_dir, image_folder='cohn-kanade-images-crop', 
                                 label_folder='Emotion', landmark_folder='Landmarks_crop',
                                 fold=x, phase='train', classes=classes, n_frames=N_frames, transform=data_transforms,
                                                 n_landmarks=N_landmarks,
                                 indexing=1)
                    for x in k_folders}

validation_datasets = {x: video_loader.VideoFolder(root=data_dir, image_folder='cohn-kanade-images-crop', 
                                 label_folder='Emotion', landmark_folder='Landmarks_crop',
                                 fold=x, phase='valid', classes=classes, n_frames=N_frames, transform=data_transforms,
                                                   n_landmarks=N_landmarks,
                                 indexing=1)
                    for x in k_folders}

fold = 0

training_folds = [x for x in range(K) if x != fold]
validation_fold = random.choice(training_folds)
training_folds = [x for x in training_folds if x is not validation_fold]

cross_datasets = {}
cross_datasets['train'] = torch.utils.data.ConcatDataset([training_datasets[k_folders[k]] 
                                                          for k in training_folds])
cross_datasets['val'] = validation_datasets[k_folders[validation_fold]]

cross_datasets['test'] = validation_datasets[k_folders[fold]]

dataloaders = {x: torch.utils.data.DataLoader(cross_datasets[x], batch_size=32,
                                              num_workers=8, shuffle=True)
              for x in ['train', 'val', 'test']}

dataset_sizes = {x: len(cross_datasets[x]) for x in ['train', 'val', 'test']}

### Oulu-CASIA

In [ ]:
use_gpu = torch.cuda.is_available()

classes =  {0:'anger', 1:'disgust', 2:'fear', 3:'happy', 4:'sadness', 5:'surprise'}
data_dir = os.path.join('/home','nii','Documents', 'OriginalImg', 'VL')
N_frames = 3
N_classes = len(classes)
N_landmarks = 68

data_transforms = transforms.Compose(
    [transforms.Resize((64,64))])

K = 10
k_folders = ['set_' + str(idx) for idx in range(K)]
    
training_datasets = {x: video_loader.VideoFolder(root=data_dir, image_folder='Strong-crop', 
                                 label_folder='Strong-emotion', landmark_folder='Landmarks_crop',
                                 fold=x, phase='train', classes=classes, img_type='jpeg', n_landmarks=N_landmarks,
                                    n_frames=N_frames, transform=data_transforms)
                    for x in k_folders}

validation_datasets = {x: video_loader.VideoFolder(root=data_dir, image_folder='Strong-crop', 
                                 label_folder='Strong-emotion', landmark_folder='Landmarks_crop',
                                 fold=x, phase='valid', classes=classes, img_type='jpeg', n_landmarks=N_landmarks,
                                      n_frames=N_frames, transform=data_transforms)
                    for x in k_folders}


fold = 6

training_folds = [x for x in range(K) if x != fold]
validation_fold = random.choice(training_folds)
training_folds = [x for x in training_folds if x is not validation_fold]

cross_datasets = {}
cross_datasets['train'] = torch.utils.data.ConcatDataset([training_datasets[k_folders[k]] 
                                                          for k in training_folds])
cross_datasets['val'] = validation_datasets[k_folders[validation_fold]]

cross_datasets['test'] = validation_datasets[k_folders[fold]]

dataloaders = {x: torch.utils.data.DataLoader(cross_datasets[x], batch_size=32,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val', 'test']}

dataset_sizes = {x: len(cross_datasets[x]) for x in ['train', 'val', 'test']}

### AFEW

In [ ]:
classes =  {0:'neutral', 1:'angry', 2:'disgust', 3:'fear', 4:'happy', 5:'sad', 6:'surprise'}
N_frames = 3
N_landmarks = 49
N_classes = len(classes)
use_gpu = torch.cuda.is_available()

# preprocessing
data_dir = os.path.join('/home','nii','Documents','EmotiW_2018','Train_AFEW')
data_dir_val = os.path.join('/home','nii','Documents','EmotiW_2018','Val_AFEW')

data_transforms = transforms.Compose(
    [transforms.Resize((64,64))])

K = 10
k_folders = ['set_' + str(idx) for idx in range(K)]   

    
training_datasets = {x: video_loader.VideoFolder(root=data_dir, image_folder='cropped_images', 
                                 label_folder='emotion', landmark_folder='landmarks',
                                 fold=x, phase='train', img_type='jpg',
                                 classes=classes, n_frames=N_frames, n_landmarks=N_landmarks,
                                 transform=data_transforms, indexing=0,
                                   are_subjects=False)
                    for x in k_folders}


validation_datasets = {x: video_loader.VideoFolder(root=data_dir, image_folder='cropped_images', 
                                 label_folder='emotion', landmark_folder='landmarks',
                                 fold=x, phase='valid', img_type='jpg',
                                 classes=classes, n_frames=N_frames,  n_landmarks=N_landmarks,
                                 transform=data_transforms, indexing=0,
                                     are_subjects=False)
                    for x in k_folders}

testing_datasets = {x: video_loader.VideoFolder(root=data_dir_val, image_folder='cropped_images', 
                                 label_folder='emotion', landmark_folder='landmarks',
                                 fold=x, phase='test', img_type='jpg',
                                 classes=classes, n_frames=N_frames,  n_landmarks=N_landmarks,
                                 transform=data_transforms, indexing=0,
                                     are_subjects=False)
                    for x in k_folders}

fold = 6

training_folds = [x for x in range(K) if x != fold]
validation_fold = fold

cross_datasets = {}
cross_datasets['train'] = torch.utils.data.ConcatDataset([training_datasets[k_folders[k]] 
                                                          for k in training_folds])
cross_datasets['val'] = validation_datasets[k_folders[validation_fold]]

cross_datasets['test'] = torch.utils.data.ConcatDataset([testing_datasets[k_folders[k]] 
                                                          for k in range(K)])

dataloaders = {x: torch.utils.data.DataLoader(cross_datasets[x], batch_size=32, shuffle=True,
                                              num_workers=4)
              for x in ['train', 'val', 'test']}

dataset_sizes = {x: len(cross_datasets[x]) for x in ['train', 'val', 'test']}

In [ ]:
videos, classes, ld = next(iter(dataloaders['val']))

## Train model

In [ ]:
def train_model(model, criterion, optimizer, num_epochs=25, min_epoch=100, k=5, alpha=3):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_loss = 1e6
    
    epoch = 0
    stop_criterion = True
    
    train_accuracy = []
    val_accuracy = []
    train_loss = []
    val_loss = []
    
    #for epoch in range(num_epochs):
    while stop_criterion:
        #print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('Epoch {}'.format(epoch))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0.0

            # Iterate over data.
            for data in dataloaders[phase]:
                # get the inputs
                _, labels, landmarks = data

                # wrap them in Variable
                if use_gpu:
                    inputs = Variable(landmarks.float().cuda())
                    labels = Variable(labels.cuda())
                else:
                    inputs, labels = Variable(landmarks.float()), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)

                loss = criterion(outputs, labels)
                
                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.data[0] * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            
            if phase == 'train':
                train_accuracy.append(epoch_acc)
                train_loss.append(epoch_loss)
            else:
                val_accuracy.append(epoch_acc)
                val_loss.append(epoch_loss)
                GL = 100 * (epoch_loss/best_loss - 1)
                Pk = 1000 * (sum(train_loss[-k:]) / (k*min(train_loss[-k:])) - 1)
                PQ = GL / Pk
                
                print('PQ = ' + str(PQ))
                
                if (PQ > alpha and epoch >= min_epoch) or epoch == num_epochs:
                    stop_criterion = False
                    
            # deep copy the model
            if phase == 'val' and epoch_acc >= best_acc:
                best_acc = epoch_acc
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
        
        epoch += 1
        
        print()
        
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, best_acc, train_accuracy, train_loss, val_accuracy, val_loss


def test_model(model, criterion):
    
    model.train(False)
    
    running_loss = 0.0
    running_corrects = 0.0
    
    truth = []
    prediction = []

    # Iterate over data.
    for data in dataloaders['test']:
        # get the inputs
        _, labels, landmarks = data

        # wrap them in Variable
        if use_gpu:
            inputs = Variable(landmarks.float().cuda())
            labels = Variable(labels.cuda())
        else:
            inputs, labels = Variable(landmarks.float()), Variable(labels)

        # forward
        outputs = model(inputs)
        _, preds = torch.max(outputs.data, 1)
        
        loss = criterion(outputs, labels)
        
        # statistics
        running_loss += loss.data[0] * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        
        truth.extend(labels.cpu().data.numpy().tolist())
        prediction.extend(preds.cpu().numpy().tolist())

    total_loss = running_loss / dataset_sizes['test']
    total_acc = running_corrects / dataset_sizes['test']

    print('{} Loss: {:.4f} Acc: {:.4f}'.format('test', total_loss, total_acc))
    
    cnf_matrix = confusion_matrix(truth, prediction, labels=list(range(N_classes)))

    return total_loss, total_acc, cnf_matrix

## Define model

Ideally this should be in its own file, for debugging purposes it was kept in the main script. The make_layers and forward functions still have some issues.

In [ ]:
# BiRNN Model (Many-to-One)
class BiRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(BiRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, 
                            batch_first=True, bidirectional=True)
    
    def forward(self, x):
        
        # Set initial states
        if x.is_cuda:
            h0 = Variable(torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).cuda()) # 2 for bidirection 
            
        else:
            h0 = Variable(torch.zeros(self.num_layers*2, x.size(0), self.hidden_size)) # 2 for bidirection 
        
        # Forward propagate RNN
        out, _ = self.rnn(x, h0)
        
        return out
    

class BiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(BiLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, 
                            batch_first=True, bidirectional=True)
    
    def forward(self, x):
        
        # Set initial states
        if x.is_cuda:
            h0 = Variable(torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).cuda()) # 2 for bidirection 
            c0 = Variable(torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).cuda())
            
        else:
            h0 = Variable(torch.zeros(self.num_layers*2, x.size(0), self.hidden_size)) # 2 for bidirection 
            c0 = Variable(torch.zeros(self.num_layers*2, x.size(0), self.hidden_size))
        
        # Forward propagate RNN
        out, _ = self.lstm(x, (h0, c0))
        
        # Decode hidden state of last time step
        #out = out[:, -1, :]
        
        return out
    

def make_layers(cfg):
    layers = []
    factor = 1
    for v in cfg:
        # extract layer configuration
        mask, hidden_size, n_layers, double, memory = v
        if memory:
            # memory layer
            #factor *= 2 # temporary correction --> multiply by the number of previous layers
            layers += [BiLSTM(len(mask)*factor, hidden_size, n_layers)]
        else:
            layers += [nn.ModuleList([BiRNN(len(mask[i])*factor, hidden_size, n_layers)
                                for i in range(len(mask))])]
            if double:
                layers += [nn.ModuleList([BiRNN(hidden_size*2, hidden_size, n_layers)
                               for i in range(len(mask))])]
        factor = 2 * hidden_size
        
    return nn.ModuleList(layers)


class Flexible(nn.Module):
    def __init__(self, cfg, n_landmarks, n_classes, use_mask=True):
        super(Flexible, self).__init__()
        
        self.n_landmarks = n_landmarks
        self.n_classes = n_classes
        self.cfg = cfg
        self.use_mask = use_mask
        
        # feature extraction
        self.features = make_layers(self.cfg)
        
        self.classifier = nn.Sequential(
            nn.Linear(480, 8192),
            nn.ReLU(True),
            nn.Linear(8192, 4096),
            nn.ReLU(True),
            nn.Linear(4096, self.n_classes),
        )
        
    
    def forward(self, x):
        
        batch_size = x.size(0)
        
        #"""
        if self.use_mask: 
            x = x.view(batch_size, -1, 2*68)
            mask = np.ones((2*68, 1),  dtype=bool)
            mask[:34] = False
            mask[120:122] = False
            mask[128:130] = False
            x = x[:,:,np.where(mask)[0]]
            
        else:
            x = x.view(batch_size, -1, 2*49)
        #"""
        
        """
        x = x.view(batch_size, -1, 2*68)
        mask = np.ones((2*68, 1),  dtype=bool)
        mask[:34] = False
        mask[120:122] = False
        mask[128:130] = False
        x = x[:,:,np.where(mask)[0]]
        """
        
        num_layer = 0
        for v in self.cfg:
            # extract layer configuration
            
            mask, hidden_size, n_layers, double, memory = v
            
            # LSTM layer
            if memory:
                x = x.contiguous().view(batch_size, n_layers, -1)
                x = self.features[num_layer](x)
                x = x.contiguous().view(batch_size, -1)
                num_layer += 1
              
            # RNN layer
            else:
                out_features = []
                for i in range(len(mask)):
                    if double:
                        landmarks = x[:,:,mask[i]].view(batch_size, n_layers, -1)
                        landmarks = self.features[num_layer+1][i](self.features[num_layer][i](landmarks))

                    else:
                        landmarks = x[:,:,mask[i]].view(batch_size, n_layers, -1)
                        landmarks = self.features[num_layer][i](landmarks)

                    out_features.append(landmarks)
                    
                if double:
                    num_layer += 2
                else:
                    num_layer += 1

                x = torch.stack(out_features).permute(1,2,0,3)
                
        # classification
        x = self.classifier(x)
        
        return x

In [ ]:
def init_weights(m):
    if type(m) == nn.Linear or type(m) == nn.Conv2d or type(m) == nn.Conv3d:
        nn.init.xavier_normal(m.weight)
        m.bias.data.fill_(0.01)

In [ ]:
del model

In [ ]:
# This is where you chose the hierachy that you want.
#arr = np.load('new_tree.npy')
#arr = manual_tree

Builds model based on hierachical tree that was given, the number are slightly arbitrary, they are mainly to reproduce the dimensions of the original PHRNN.

In [ ]:
my_config = []
for layer in range(arr.shape[1] - 1):
    
    # if not last layer
    max_cluster = np.max(arr[:, layer+1]) + 1 
    masks = []
    # get mask for all cluster
    for cluster in range(max_cluster):
        masks.append(np.unique(arr[np.equal(arr[:,layer+1], cluster), layer]))
    
    if max_cluster <= 4:
        double = True
    else:
        double = False
    
    # if last layer
    if layer == arr.shape[1] - 2:
        memory = True
        h_size = 80
        
    else:
        memory = False
        h_size = int(450/(1 + 2*max_cluster))
        
    my_config.append((masks, h_size, 3, double, memory))
    
model = Flexible(my_config, 49, N_classes, use_mask=True)
model.apply(init_weights)

Other configurations, run this script instead if you want to compare with the original PHRNN.

In [ ]:
init_masks = [np.array(range(34,71+1)), # eyebrows + nose
              np.array(range(72,83+1)), # left eye
              np.array(range(84,95+1)), # right eye
              np.concatenate([np.array(range(96,109+1)), np.array(range(120,129+1))]), # upper lip
              np.concatenate([np.array(range(110,119+1)), np.array(range(130,135+1))]) # lower lip
             ]

init_masks_kao = [np.array(range(34,53+1)), # eyebrows
                  np.array(range(72,95+1)), # eyes
                  np.array(range(54,71+1)), # nose
                  np.array(range(96,135+1)) # mouth
                 ]

# for 49 landmarks
init_masks_kao = [np.array(range(0,20)), # eyebrows
                  np.array(range(38,60)), # eyes
                  np.array(range(20,38)), # nose
                  np.array(range(60,98)) # mouth
                 ]

# for each layer : mask/join, hidden_size, length of sequence, is double ?, has memory ?
my_config = [
       (init_masks, 30, 3, False, False),
       ([[0],[1,2],[3,4]], 60, 3, True, False),
       ([[0,1],[0,2]], 90, 3, True, False),
       ([[0,1],[0,2]], 80, 3, False, True)
]

kao_config = [
    (init_masks_kao, 30, 3, False, False),
    ([[0,1],[2],[3]], 60, 3, True, False),
    ([[0,1],[1,2]], 90, 3, True, False),
    ([[0,1],[1,2]], 80, 3, False, True)
]

model = Flexible(kao_config, 49, N_classes)
model.apply(init_weights)

In [ ]:
parameters = model.parameters()

if use_gpu:
    model = model.cuda()
    
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(parameters, lr=0.001, weight_decay=5e-5)

In [ ]:
model, _, train_acc, train_loss, val_acc, val_loss = train_model(model, criterion, optimizer, k=20, alpha=1.5, 
                                                                 num_epochs=1000, min_epoch=200)

In [ ]:
test_loss, test_accuracy = test_model(model, criterion)

In [ ]:
save_folder = 'model'

As of pytorch 0.3.1, it is not possible to save bidirectionnal RNN hidden states weights with the regular save function, thus the whole model needs to be saved altogether. This means that loading the model in other scripts will require the source code of the original model.

In [ ]:
save_path = os.path.join(save_folder, 'manual_' + str(fold) + '.pt')
torch.save(model, save_path)

In [ ]:
del bilstm

In [ ]:
bilstm = torch.load(os.path.join(save_folder, 'test_' + str(fold) + '.pt'))

if use_gpu:
    bilstm = bilstm.cuda()

In [ ]:
test_loss, test_accuracy = test_model(model, criterion)

## Plot accuracy and loss

To better visualize the accuracy/loss, it is recommended to take the moving average instead.

In [ ]:
n_epochs = len(train_acc)
window_width = 20

cumsum_vec = np.cumsum(np.insert(train_acc, 0, 0)) 
ma_vec_train = (cumsum_vec[window_width:] - cumsum_vec[:-window_width]) / window_width

cumsum_vec = np.cumsum(np.insert(val_acc, 0, 0)) 
ma_vec_val = (cumsum_vec[window_width:] - cumsum_vec[:-window_width]) / window_width

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(range(window_width,n_epochs+1), ma_vec_train, color='r', label='training')
plt.plot(range(window_width,n_epochs+1), ma_vec_val, color='b', label='validation')
#plt.plot(range(n_epochs), train_acc,color='r', label='training')
#plt.plot(range(n_epochs), val_acc,color='b', label='validation')
plt.grid(color='k', linestyle='-', linewidth=1)
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.title('PHRNN training')
pylab.legend(loc='lower right')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
#plt.plot(range(window_width,1001), ma_vec_train, color='r', label='training')
#plt.plot(range(window_width,1001),ma_vec_val, color='b', label='validation')
plt.plot(range(n_epochs), train_loss, color='r', label='training')
plt.plot(range(n_epochs), val_loss, color='b', label='validation')
plt.grid(color='k', linestyle='-', linewidth=1)
plt.xlabel('epochs')
plt.ylabel('loss')
plt.title('PHRNN training')
pylab.legend(loc='upper right')
plt.tight_layout()
plt.show()

In [ ]:
# save model
save_folder = 'model'
save_path = os.path.join(save_folder, 'phrnn.pt')
torch.save(model.state_dict(), save_path)

## K-fold crossvalidation

In [ ]:
k_accuracy = []
save_folder = 'model'

for fold in range(K):
    
    print('Starting fold ' + str(fold) + ' ...')
    
    # reassign datasets to training, validation and testing
    training_folds = [x for x in range(K) if x is not fold]
    validation_fold = random.choice(training_folds)
    training_folds = [x for x in training_folds if x is not validation_fold]
    
    cross_datasets = {}
    cross_datasets['train'] = torch.utils.data.ConcatDataset([training_datasets[k_folders[k]] 
                                                              for k in training_folds])
    cross_datasets['val'] = validation_datasets[k_folders[validation_fold]]
    
    cross_datasets['test'] = validation_datasets[k_folders[fold]]

    dataloaders = {x: torch.utils.data.DataLoader(cross_datasets[x], batch_size=32,
                                                 shuffle=True, num_workers=8)
                  for x in ['train', 'val', 'test']}
    dataset_sizes = {x: len(cross_datasets[x]) for x in ['train', 'val', 'test']}
    
    # init model
    model = Flexible(kao_config, 49, N_classes, use_mask=True)
    model.apply(init_weights)
    
    # set up optimizer
    parameters = model.parameters()

    if use_gpu:
        model = model.cuda()

    optimizer = optim.Adamax(parameters, lr=0.001)

    # train model
    model, val_accuracy, train_acc, train_loss, val_acc, val_loss = train_model(model, criterion, optimizer, 
                                                                                k=20, alpha=1.5, num_epochs=1000, min_epoch=200)
    test_loss, test_accuracy, conf = test_model(model, criterion)
    
    # save model
    save_path = os.path.join(save_folder, 'kaohai_oulu' + str(fold) + '.pt')
    #torch.save(model.state_dict(), save_path)
    torch.save(model, save_path)
    del model
    
    
    print('Finished fold ' + str(fold) + ' with validation accuracy of ' + str(val_accuracy))
    k_accuracy.append(test_accuracy)

print('Mean value of test accuracy over ' + str(K) + '-fold crossvalidation is: ' 
      + str(sum(k_accuracy) / float(len(k_accuracy))))

In [ ]:
k_accuracy

In [ ]:
plt.boxplot(k_accuracy)

In [ ]:
del model

In [ ]:
model

### Cross-validation test

In [ ]:
k_accuracy = []
save_folder = 'model'
conf_math = np.zeros((N_classes, N_classes))

for fold in range(K):
    print('Starting fold ' + str(fold) + ' ...')
    
    # reassign datasets to training, validation and testing
    training_folds = [x for x in range(K) if x != fold]
    validation_fold = random.choice(training_folds)
    training_folds = [x for x in training_folds if x is not validation_fold]
    
    cross_datasets = {}
    cross_datasets['train'] = torch.utils.data.ConcatDataset([training_datasets[k_folders[k]] 
                                                              for k in training_folds])
    cross_datasets['val'] = validation_datasets[k_folders[validation_fold]]
    
    cross_datasets['test'] = validation_datasets[k_folders[fold]]

    dataloaders = {x: torch.utils.data.DataLoader(cross_datasets[x], batch_size=64,
                                                 shuffle=True, num_workers=4)
                  for x in ['train', 'val', 'test']}
    dataset_sizes = {x: len(cross_datasets[x]) for x in ['train', 'val', 'test']}
        
    model = torch.load(os.path.join(save_folder, 'manual_oulu' + str(fold) + '.pt'))
    model.eval()
    
    if use_gpu:
        model = model.cuda()
    
    test_loss, test_accuracy, conf = test_model(model, criterion)
    conf_math += conf
    k_accuracy.append(test_accuracy)
    
    del model
    
print('Mean value of test accuracy over ' + str(K) + '-fold cr_ossvalidation is: ' 
      + str(sum(k_accuracy) / float(len(k_accuracy))))

In [ ]:
conf_math

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    #plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
list(classes.values())

In [ ]:
# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(conf_math, classes=list(classes.values()), normalize=True,
                      title='Normalized confusion matrix')

plt.show()

In [ ]:
k_accuracy